# 📁 01 - Data Handling

### 🎯 Objective
This notebook is responsible for loading, cleaning, and merging the raw CSV data exported from my personal **Samsung Health™** account. The result is a single, unified dataset ready for downstream time series processing and machine learning tasks.

---

### 💻 Devices Used
- **Samsung Galaxy Note 9** (Until July 2022)  
- **Samsung Galaxy S22 Ultra + Samsung Watch 4 Classic** (After July 2022)

> **Samsung is a registered trademark of Samsung Electronics Co., Ltd.**  
> This project is for personal and educational purposes only.

---

### 🛠️ Key Steps

1. **Load Raw Files**  
   Import all CSVs exported from Samsung Health — each containing data from different modalities (nutrition, exercise, sleep, stress, etc.).

2. **Clean Columns**  
   - Remove unnecessary rows (e.g., metadata or malformed headers)  
   - Rename and standardize column names and formats  
   - Convert datatypes as needed

3. **Initial Checks**  
   - Explore missing values  
   - Identify inconsistent date ranges or sparsity

4. **Merge Datasets**  
   Join all dataframes on the `date` column, ensuring features are aligned chronologically.

5. **Save Intermediate Output**  
   Export the merged, lightly cleaned dataset to `data.csv`, for reuse in subsequent notebooks.

---

### 📦 Output
- `data.csv`: Unified daily time series dataset with features from most of the health modalities.

---

> 📝 **Note**: This notebook performs only structural and light cleaning. Full feature engineering and normalization are handled in later steps of the pipeline.
> ⚠️ **Note on Data Availability**:
The quantity and type of variables available may vary depending on your devices and how they are set up. You might need to adapt parts of this project to match the structure and availability of your own Samsung Health data.


In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 1000) #Display all the columns.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import chardet for Character detection

# File name format:
# - %Y%m%d means date in format like 01012025
# - XXXXXX I think it's like a personal ID so check it on your files.

# with open("../data/raw/com.samsung.health.floors_climbed.%Y%m%dXXXXXX.csv", "rb") as f:
#     raw = f.read(10000)
#     result = chardet.detect(raw)
#     print(result)
#     #{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''} <= Compatible with latin-1

In [ ]:
floors = pd.read_csv("../data/raw/com.samsung.health.floors_climbed.%Y%m%dXXXXXX.csv",
                        sep=",",
                        header=1, #Avoid the first row of the file. 
                        index_col=False, # Force pandas to not use the first column as the index.
                        encoding="latin-1", #UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 28: invalid continuation byte.
                        na_values=["", " ", "NaN", "nan"], # Additional strings to recognize as NA/NaN.
                        keep_default_na=True)

In [ ]:
floors.head(10)

In [ ]:
floors.columns

In [ ]:
# I'll keep only one date for reference and future data mergings.
floors.drop(columns=['custom', 'update_time', 'create_time', 'client_data_id',
       'client_data_ver','raw_data', 'time_offset', 'deviceuuid',
       'pkg_name', 'end_time','datauuid'], inplace=True)

In [ ]:
floors.dtypes

In [ ]:
#I'll format the date as day format:
floors['start_time'] = pd.to_datetime(floors['start_time']).dt.date

In [ ]:
floors.head()

In [ ]:
#The idea is that, keep a row per day so:
floors = floors.groupby('start_time')['floor'].sum().reset_index()
#IMPORTANT: groupby returns DataFrameGroupBy. After applying an aggregation functions (like sum()) returns a pandasSeries. 
# Only applying reset_index you finally get a DataFrame. 

In [ ]:
floors.head()

In [ ]:
# Renaming the columns for future clarity:
floors.columns = ['date', 'total_floors_climbed']

---

In [ ]:
food_info = pd.read_csv("../data/raw/com.samsung.health.food_info.%Y%m%dXXXXXX.csv", 
                        sep=",",
                        header=1, # Python index from zero so avoid the first row and take the second one as header.
                        index_col=False, # Force pandas to not use the first column as the index.
                        encoding="latin-1", #UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 28: invalid continuation byte.
                        na_values=["", " ", "NaN", "nan"], # Additional strings to recognize as NA/NaN.
                        keep_default_na=True)

In [ ]:
food_info.head()

In [ ]:
food_info.columns

In [ ]:
# Most of the values are 0.0 perhaps because this feature 
# it's not informed by the food industry so we need to remove it:
food_info['added_sugar'].value_counts()

In [ ]:
food_info.drop(columns=['custom', 'provider_food_id',
                        'update_time',
                        'serving_description',
                        'info_provider', 'deviceuuid',
                        'metric_serving_unit',
                        'pkg_name', 'unit_count_per_calorie',
                        'default_number_of_serving_unit','description',
                        'metric_serving_amount', 'name',
                        'added_sugar', 'datauuid'
                        ], inplace=True)

In [ ]:
food_info['create_time'] = pd.to_datetime(food_info['create_time']).dt.date

In [ ]:
food_info.dtypes

In [ ]:
food_info.head()

In [ ]:
food_info.tail()

In [ ]:
# Perhaps this feature it's not useful at all neither (Fully of 0.0 values)
food_info['vitamin_d'].value_counts()

In [ ]:
food_info.drop(columns='vitamin_d', inplace=True)

In [ ]:
# This one can keep:
food_info['trans_fat'].value_counts()

In [ ]:
food_info = food_info.groupby('create_time').sum().reset_index()
food_info.head()

In [ ]:
food_info.columns

In [ ]:
# Here again, renaming the columns for future clarity:
food_info.columns = ['date', 'potassium', 'vitamin_a', 'vitamin_c', 'cholesterol',
       'sodium', 'dietary_fiber', 'total_fat', 'monosaturated_fat', 'protein',
       'polysaturated_fat', 'iron', 'sugar', 'calcium', 'ingested_calories',
       'saturated_fat', 'trans_fat', 'carbohydrate']

---

In [ ]:
weight = pd.read_csv("../data/raw/com.samsung.health.weight.%Y%m%dXXXXXX.csv",
                        sep=",",
                        header=1,
                        index_col=False, # Force pandas to not use the first column as the index
                        encoding="latin-1", #UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 28: invalid continuation byte
                        na_values=["", " ", "NaN", "nan"], # Additional strings to recognize as NA/NaN
                        keep_default_na=True)

In [ ]:
weight.tail()

In [ ]:
weight.columns

In [ ]:
weight.drop(columns=['create_sh_ver', 'custom','modify_sh_ver', 
                     'update_time', 'create_time','client_data_id',
                     'client_data_ver','deviceuuid', 'time_offset',
                     'comment','pkg_name','vfa_level','datauuid'], 
            inplace=True)

In [ ]:
weight['start_time'] = pd.to_datetime(weight['start_time']).dt.date

In [ ]:
weight.dtypes

In [ ]:
weight.head()

In [ ]:
weight.tail()

In [ ]:
# This feature will be removed:
weight['muscle_mass'].value_counts()

In [ ]:
weight.drop(columns='muscle_mass', inplace=True)

In [ ]:
# This one we can keep it:
weight['skeletal_muscle'].value_counts()

In [ ]:
weight.columns

In [ ]:
# Column renaming:
weight.columns = ['body_fat_mass', 'date', 'height', 'weight', 'skeletal_muscle',
       'fat_free_mass', 'basal_metabolic_rate', 'skeletal_muscle_mass',
       'fat_free', 'body_fat', 'total_body_water']

---

In [ ]:
activity = pd.read_csv("../data/raw/com.samsung.shealth.activity.day_summary.%Y%m%dXXXXXX.csv",
                        sep=",",
                        header=1,
                        index_col=False, # Force pandas to not use the first column as the index
                        encoding="latin-1", #UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 28: invalid continuation byte
                        na_values=["", " ", "NaN", "nan"], # Additional strings to recognize as NA/NaN
                        keep_default_na=True)

In [ ]:
activity.tail()

In [ ]:
activity.columns

In [ ]:
activity.drop(columns=['create_sh_ver', 'energy_type', 'exercise_time',
                        'exercise_calorie_target', 'active_time', 'target',
                        'others_time', 'modify_sh_ver','update_time','floors_target',
                        'create_time', 'floor_count', 'dynamic_active_time_target',
                        'exercise_time_target', 'goal', 'longest_active_time',
                         'move_hourly_count', 'duration_type', 'move_hourly_target', 'distance',
                        'dynamic_active_time','extra_data', 'deviceuuid',
                        'run_time', 'pkg_name', 'walk_time', 'longest_idle_time','datauuid'], inplace=True)

In [ ]:
activity.dtypes

In [ ]:
activity['day_time']=pd.to_datetime(activity['day_time']).dt.date

In [ ]:
activity.head()

In [ ]:
activity = activity.groupby('day_time').sum().reset_index()

In [ ]:
activity.head()

In [ ]:
activity.tail()

In [ ]:
# This feature need to be removed (All values equal to 0.0):
activity['movement_type'].value_counts()

In [ ]:
activity.drop(columns='movement_type', inplace=True)

In [ ]:
activity.columns

In [ ]:
# Renaming columns:
activity.columns = ['date', 'step_count', 'score', 'total_steps_burned_calories']

---

In [ ]:
burned =  pd.read_csv("../data/raw/com.samsung.shealth.calories_burned.details.%Y%m%dXXXXXX.csv",
                        sep=",",
                        header=1,
                        index_col=False, # Force pandas to not use the first column as the index
                        encoding="latin-1", #UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 28: invalid continuation byte
                        na_values=["", " ", "NaN", "nan"], # Additional strings to recognize as NA/NaN
                        keep_default_na=True)

In [ ]:
burned.tail()

In [ ]:
burned.columns

In [ ]:
burned.drop(columns=['active_calories_goal', 'version', 'extra_data', 'exercise_calories',
                     'com.samsung.shealth.calories_burned.create_sh_ver',
                    'com.samsung.shealth.calories_burned.modify_sh_ver',
                    'com.samsung.shealth.calories_burned.update_time',
                    'com.samsung.shealth.calories_burned.active_calorie',
                    'com.samsung.shealth.calories_burned.deviceuuid',
                    'com.samsung.shealth.calories_burned.pkg_name',
                    'com.samsung.shealth.calories_burned.day_time',
                    'com.samsung.shealth.calories_burned.datauuid'], inplace=True)

In [ ]:
burned['com.samsung.shealth.calories_burned.create_time'] = pd.to_datetime(burned['com.samsung.shealth.calories_burned.create_time']).dt.date

In [ ]:
burned.head()

In [ ]:
burned.tail()

In [ ]:
burned.columns = ['total_excercise_calories', 'burned_tef_calories', 'burned_active_time',
                  'burned_rest_calories', 'date']

---

In [ ]:
sleep =  pd.read_csv("../data/raw/com.samsung.shealth.sleep_combined.%Y%m%dXXXXXX.csv",
                        sep=",",
                        header=1,
                        index_col=False, # Force pandas to not use the first column as the index
                        encoding="latin-1", #UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 28: invalid continuation byte
                        na_values=["", " ", "NaN", "nan"], # Additional strings to recognize as NA/NaN
                        keep_default_na=True)

In [ ]:
sleep.tail()

In [ ]:
# Check the feature column to see if it's a real NaN feature or not at all. 
sleep['latency_weight'].isna().value_counts()

In [ ]:
sleep.columns

In [ ]:
sleep.drop(columns=['total_sleep_time_weight', 'original_efficiency', 'create_sh_ver',
       'start_time','wake_score', 'deep_score', 'latency_weight',
       'has_sleep_data', 'sleep_efficiency_with_latency','total_rem_duration',
       'modify_sh_ver', 'update_time', 'create_time', 'client_data_id',
       'sleep_type', 'data_version', 'latency_score', 'deep_weight',
       'rem_weight','original_wake_up_time',
       'client_data_ver','original_bed_time',
       'goal_bed_time', 'quality', 'time_offset', 'extra_data', 'wake_weight',
       'deviceuuid', 'rem_score', 'goal_wake_up_time', 'sleep_cycle','pkg_name',
        'stage_analyzed_type','stage_analysis_type', 'total_sleep_time_score','datauuid',
        ], inplace=True)

In [ ]:
sleep.head()

In [ ]:
sleep.tail()

In [ ]:
# This feature must be removed:
sleep['factor_10'].isna().value_counts()

In [ ]:
sleep.drop(columns='factor_10', inplace=True)

In [ ]:
sleep['end_time'] = pd.to_datetime(sleep['end_time']).dt.date

In [ ]:
sleep.tail()

In [ ]:
sleep = sleep.add_prefix('sleep_')

In [ ]:
sleep.columns

In [ ]:
sleep.columns = ['sleep_mental_recovery', 'sleep_factor_01', 'sleep_factor_02',
       'sleep_factor_03', 'sleep_factor_04', 'sleep_factor_05',
       'sleep_factor_06', 'sleep_factor_07', 'sleep_factor_08',
       'sleep_factor_09', 'sleep_physical_recovery',
       'sleep_movement_awakening', 'sleep_total_light_duration',
       'sleep_efficiency', 'sleep_sleep_score', 'sleep_sleep_duration',
       'date']

---

In [ ]:
stress =  pd.read_csv("../data/raw/com.samsung.shealth.stress.%Y%m%dXXXXXX.csv",
                        sep=",",
                        header=1,
                        index_col=False, # Force pandas to not use the first column as the index
                        encoding="latin-1", #UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 28: invalid continuation byte
                        na_values=["", " ", "NaN", "nan"], # Additional strings to recognize as NA/NaN
                        keep_default_na=True)

In [ ]:
stress.tail()

In [ ]:
stress.columns

In [ ]:
stress.drop(columns=['create_sh_ver', 'start_time', 'custom', 'binning_data', 
                     'tag_id','modify_sh_ver', 'update_time', 'create_time', 
                     'algorithm', 'time_offset', 'deviceuuid', 'comment', 
                     'pkg_name','datauuid'], inplace=True)

In [ ]:
stress['end_time'] = pd.to_datetime(stress['end_time']).dt.date

In [ ]:
stress.tail()

In [ ]:
# If you want to add a prefix or suffix to every column name, 
# you can use the add_prefix() and add_suffix() methods. 
# These methods are helpful when you need to modify all column names in a consistent way.
stress = stress.add_prefix('stress_')

In [ ]:
stress.head()

In [ ]:
stress = stress.groupby('stress_end_time').sum().reset_index()

In [ ]:
stress.tail()

In [ ]:
stress = stress.rename(columns={'stress_end_time': 'date'})

In [ ]:
stress.columns

---

In [ ]:
heart =  pd.read_csv("../data/raw/com.samsung.shealth.tracker.heart_rate.%Y%m%dXXXXXX.csv",
                        sep=",",
                        header=1,
                        index_col=False, # Force pandas to not use the first column as the index
                        encoding="latin-1", #UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 28: invalid continuation byte
                        na_values=["", " ", "NaN", "nan"], # Additional strings to recognize as NA/NaN
                        keep_default_na=True)

In [ ]:
heart.tail()

In [ ]:
heart.columns

In [ ]:
heart.drop(columns=['source', 'tag_id', 'com.samsung.health.heart_rate.create_sh_ver',
       'com.samsung.health.heart_rate.heart_beat_count',
       'com.samsung.health.heart_rate.start_time',
       'com.samsung.health.heart_rate.custom',
       'com.samsung.health.heart_rate.binning_data',
       'com.samsung.health.heart_rate.modify_sh_ver',
       'com.samsung.health.heart_rate.update_time',
       'com.samsung.health.heart_rate.create_time',
       'com.samsung.health.heart_rate.client_data_id',
        'com.samsung.health.heart_rate.client_data_ver',
       'com.samsung.health.heart_rate.time_offset',
       'com.samsung.health.heart_rate.deviceuuid',
       'com.samsung.health.heart_rate.comment',
       'com.samsung.health.heart_rate.pkg_name',
       'com.samsung.health.heart_rate.datauuid'
       ], inplace=True)

In [ ]:
heart['com.samsung.health.heart_rate.end_time'] = pd.to_datetime(heart['com.samsung.health.heart_rate.end_time']).dt.date

In [ ]:
heart.head()

In [ ]:
heart.tail()

In [ ]:
heart.dtypes

In [ ]:
heart.isna().sum()

In [ ]:
heart.shape

In [ ]:
# 2. Sort by time (VERY IMPORTANT)
heart = heart.sort_index()

# 3. Impute NaNs with the previous valid value
heart = heart.bfill().ffill()

In [ ]:
heart.isna().sum()

In [ ]:
# Here I need to take the max, min and the median value of the heart rate for each date:
df1 = heart.groupby('com.samsung.health.heart_rate.end_time')['com.samsung.health.heart_rate.max'].max().reset_index()
df2 = heart.groupby('com.samsung.health.heart_rate.end_time')['com.samsung.health.heart_rate.min'].min().reset_index()
df3 = heart.groupby('com.samsung.health.heart_rate.end_time')['com.samsung.health.heart_rate.heart_rate'].median().reset_index()

heart = pd.merge(df1, df2, on='com.samsung.health.heart_rate.end_time', how='outer') # Use 'outer' to keep all dates
heart = pd.merge(heart, df3, on='com.samsung.health.heart_rate.end_time', how='outer')

In [ ]:
heart.head()

In [ ]:
heart.columns = ['date', 'heart_max_rate', 'heart_min_rate','heart_rate']

In [ ]:
heart.head()

---

In [ ]:
data = pd.merge(floors, food_info, on='date', how='outer')
data = pd.merge(data, weight, on='date', how='outer')
data = pd.merge(data, activity, on='date', how='outer')
data = pd.merge(data, burned, on='date', how='outer')
data = pd.merge(data, sleep, on='date', how='outer')
data = pd.merge(data, stress, on='date', how='outer')
data = pd.merge(data, heart, on='date', how='outer')


In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
#Check for date duplicates just in case:
data.duplicated(subset='date').sum()

In [ ]:
# .sum() it's the most straightforward but perhaps it's not suitable if the duplicates comes from weight, stress or heart rate.
# In this cases will be better take the mean or median instead.
# Check your case before apply it. 
data = data.groupby('date').sum().reset_index()

#### Imputation Process
##### We will try to do it with rationality and common sense in order to avoid lose a lot of data

In [ ]:
data.columns

In [ ]:
#On body composition, I'll assume that, the last state is the same as the previews so:
cols = ['body_fat_mass', 'height', 'weight', 'skeletal_muscle', 'fat_free_mass',
       'basal_metabolic_rate', 'skeletal_muscle_mass', 'fat_free', 'body_fat',
       'total_body_water']

#Here first need to convert the zero coerced values to NaN in order to then back fill them.
data[cols] = data[cols].replace(0.0, np.nan)

data[cols] = data[cols].bfill()

In [ ]:
#Steps imputation. Strictly rationality so:
cols = ['burned_tef_calories', 'burned_active_time','burned_rest_calories']

data[cols] = data[cols].replace(0.0, np.nan)

#These features I think are independents of the steps so:
data[cols] = data[cols].fillna(value=data[cols].median())

In [ ]:
# Here I think the NaN was generate by the watch during my sleeping time so I'll asumme that was similar to the last record:
cols = ['sleep_mental_recovery', 'sleep_factor_01',
       'sleep_factor_02', 'sleep_factor_03', 'sleep_factor_04',
       'sleep_factor_05', 'sleep_factor_06', 'sleep_factor_07',
       'sleep_factor_08', 'sleep_factor_09', 'sleep_physical_recovery',
       'sleep_movement_awakening', 'sleep_total_light_duration',
       'sleep_efficiency', 'sleep_sleep_score', 'sleep_sleep_duration']

data[cols] = data[cols].replace(0.0, np.nan)

data[cols] = data[cols].bfill()

In [ ]:
# Same for stress and heart rate:
cols = ['stress_max', 'stress_min', 'stress_score', 'heart_max_rate',
       'heart_min_rate', 'heart_rate']

data[cols] = data[cols].replace(0.0, np.nan)

data[cols] = data[cols].bfill()

In [ ]:
data.tail()

In [ ]:
# We need to trim the dataset from July 3rd 2022 in order to eliminate the bad backward fill imputation.
data['date'] = data['date'].astype('str')
data[data['date'] > '2022-07-01']['stress_score']

In [ ]:
data = data.loc[571:,:]

In [ ]:
# This starting file will be use in EDA file in order to analyze:
data.to_csv('../data/processed/data.csv', index=False)